# Imports

In [1]:
# %load_ext nb_black
# %config Completer.use_jedi = False
import nest_asyncio
# from IPython.core.interactiveshell import InteractiveShell

# nest_asyncio.apply()
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
from lazyft import hyperopt, backtest, remote
from lazyft.quicktools.quick_tools import Config
from rich import console, pretty, traceback
from lazyft.command_parameters import HyperoptParameters, BacktestParameters
from lazyft.reports import get_backtest_repo, get_hyperopt_repo
from lazyft.models import Strategy
import pandas as pd
from lazyft.quicktools.quick_tools import QuickTools
c = console.Console(width=300, soft_wrap=True)
print = c.print
pretty.install(console=c)
traceback.install(console=c, show_locals=True)

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f0444ed0cd0>>

2021-09-15 21:50:07.076 | INFO     | lazyft.util:set_params_file:40 - Copying id file 94db2236-0861-4d7b-b720-7ecd04999290.json to strategy folder
2021-09-15 21:50:07.078 | INFO     | lazyft.strategy:create_strategy_params_filepath:87 - Getting parameters file of NotAnotherSMAOffsetStrategyModHO
2021-09-15 21:50:08.747 | INFO     | lazyft.strategy:create_strategy_params_filepath:87 - Getting parameters file of NASOSv4
2021-09-15 21:50:10.400 | INFO     | lazyft.util:remove_params_file:75 - Removing strategy params: /home/sage/ftworkdir/user_data/strategies/NASOSv4.json
2021-09-15 21:50:10.402 | INFO     | lazyft.strategy:create_strategy_params_filepath:87 - Getting parameters file of NostalgiaForInfinityNext
2021-09-15 21:50:12.052 | INFO     | lazyft.util:remove_params_file:75 - Removing strategy params: /home/sage/ftworkdir/user_data/strategies/NostalgiaForInfinityNext.json
2021-09-15 21:50:12.053 | INFO     | lazyft.util:set_params_file:40 - Copying id file 92435106-95f4-4e78-bc4e-3

# Configuration

In [3]:
# chosen_conf = Config("config_binance.com.json")
# chosen_conf = Config('config_binance.us.json')
private_conf = Config("binancecom.json")
us_conf = Config("config_binance.us.json")
kucoin = Config("config_3_100_unlimited_usdt.json")
# us_conf_no_blacklist = Config("config_binance_no_bl.us.json")
int_conf = Config("config_binance.com.json")
priv_us = Config("binanceus.json")
priv_int = Config("binancecom.json")

# chosen_conf = config_com
# general
n_coins = 50
starting_balance = 100
max_open_trades = 4
stake_amount = starting_balance / max_open_trades
interval = "5m"
inf_interval = "1h"
# config_new['timeframe'] = interval
timerange = "20210601-20210701"
timerange = None

# h_strategies = ["CombinedBinHClucAndMADV9", "BollingerBands3"]
# strategies = [
#     "TTF",
#     "BollingerBands2",
#     "BollingerBands3",
#     "NFI5MOHO_WIP",
#     "NotAnotherSMAOffsetStrategyModHO-yiXgyD",
# ]

# Data Downloading

In [ ]:
QuickTools.download_data(kucoin, '5m 1h', 10, verbose=True)

# Hyperopting

In [ ]:
days = 365
# VRA/USDT HTR/USDT ARX/USDT
params = HyperoptParameters(
    epochs=400,
    strategies=[
        "NASOSv4",
#         Strategy(id='b40294bf-de3b-4f7d-a6f7-a412709e549c')
               ],
    config_path=kucoin,
    #     secrets_config=str(priv_us),
    days=days,
#     timerange="20200701-20201231",
    spaces="default",
    loss='ROIAndProfitHyperOptLoss',
    interval="5m",
    inf_interval="30m 1h",
    min_trades=121,
    pairs=kucoin.whitelist,
#     pairs=['KSM/USDT'],
    #     pairs=pairlist.load_pairlist_from_id("52V2DPay"),
    starting_balance=starting_balance,
    max_open_trades=3,
    jobs=-2,
    download_data=False,
    tag=""
)
params
# QuickTools.refresh_pairlist(us_conf, 100)

In [ ]:
h_commands = hyperopt.create_commands(params, verbose=False)
h_runner = hyperopt.HyperoptRunner(h_commands[0])
h_runner.execute(background=True)

In [ ]:
h_runner.get_results()

In [ ]:
h_runner.running

In [ ]:
h_runner.live_output()

In [ ]:
_ = !freqtrade hyperopt-show --best
report = h_runner.generate_report()
report.save()

In [ ]:
h_runner.stop()

In [ ]:
print(h_runner.output)
# regex.EPOCH_LINE_REGEX.findall(h_runner.output)

# Backtesting

In [4]:
# b_strategies = [
#     f"{s}-{id}"
#     for s, id in get_hyperopt_repo()
#     .sort_by_loss()
#     .head(3)
#     .get_strategy_id_pairs()
# ]
b_strategies = [
    "EnsembleStrategy"
#     "NotAnotherSMAOffsetStrategyHOv3Mod",
#     "ElliotV8_original_ichiv2",
#     "Elliotv8",
#     "NASOSv4",
#     "NASOSv5",
#     "DIV_v1",
#     "BurkoBB1Strategy"
#     "TAD",
#     "Diamond",
#     "ElliotV3",
#     "Elliotv8",
#     "BigZ07",
#     "CombinedBinHClucAndMADV9",
#     "MADisplaceV3",
#     "SMAOG",
#     "Combined_NFIv7_SMA",
#     "NotAnotherSMAOffsetStrategyModHO",
#     "NotAnotherSMAOffsetStrategyHOv3",
#     "NFI5MOHO_WIP",
#     "TDLA",
#     "NostalgiaForInfinityV7",
#     "NotAnotherSMAOffsetStrategy_UZI",
#     "NotAnotherSMAOffSetStrategy_V2",
#     Strategy(id='7d818358-d86d-4301-9282-8aa32ecf359a'),
#     Strategy(id='b40294bf-de3b-4f7d-a6f7-a412709e549c'),
#     Strategy(id='ca1f7c9b-e3d6-483b-b912-00688410f0ed')
]
ensemble=[
    "NotAnotherSMAOffsetStrategyModHO-94db2236-0861-4d7b-b720-7ecd04999290",
    "NASOSv4",
    "NostalgiaForInfinityNext",
    "NotAnotherSMAOffsetStrategyHOv3Mod-92435106-95f4-4e78-bc4e-3f7ec1509f36"
         ]
b_timerange = "20210820-"
# b_timerange = None
b_days = 365
b_interval = "5m 1h"
b_conf = kucoin
# b_conf = kucoin
b_params = BacktestParameters(
    strategies=b_strategies,
    interval="5m",
    inf_interval="30m 1h",
    config_path=b_conf,
    days=b_days,
    pairs=b_conf.whitelist,
#     pairs=['VRA/USDT', 'HTR/USDT', 'KSM/USDT'],
    stake_amount="unlimited",
    starting_balance=100,
    max_open_trades=3,
    timerange=b_timerange,
    download_data=False,
    tag='',
    ensemble=ensemble
)
b_params

BacktestParameters(
    config_path='/home/sage/ftworkdir/configs/config_3_100_unlimited_usdt.json',
    secrets_config=None,
    strategies=['EnsembleStrategy'],
    download_data=False,
    ensemble=[
        Strategy(name='NotAnotherSMAOffsetStrategyModHO', id='94db2236-0861-4d7b-b720-7ecd04999290'),
        Strategy(name='NASOSv4', id=None),
        Strategy(name='NostalgiaForInfinityNext', id=None),
        Strategy(name='NotAnotherSMAOffsetStrategyHOv3Mod', id='92435106-95f4-4e78-bc4e-3f7ec1509f36')
    ],
    timerange='20210820-',
    pairs=[
        'ADA/USDT',
        'XRP/USDT',
        'LUNA/USDT',
        'CIRUS/USDT',
        'DOGE/USDT',
        'AVAX/USDT',
        'VET/USDT',
        'VRA/USDT',
        'DOT/USDT',
        'KCS/USDT',
        'QI/USDT',
        'COTI/USDT',
        'SOL/USDT',
        'MASK/USDT',
        'DAG/USDT',
        'ICP/USDT',
        'LABS/USDT',
        'EOS/USDT',
        'LINK/USDT',
        'MATIC/USDT',
        'XLM/USDT',
        'LTC/USDT',
        'XTZ/USDT',
        'SAND/USDT',
        'ZEC/USDT',
        'XDC/USDT',
        'QNT/USDT',
        'HTR/USDT',
        'TRX/USDT',
        'THETA/USDT',
        'ALGO/USDT',
        'KSM/USDT',
        'GRT/USDT',
        'AAVE/USDT',
        'CHR/USDT',
        'FTM/USDT',
        'XAVA/USDT',
        'ERG/USDT',
        'CAKE/USDT',
        'SHA/USDT',
        'MIR/USDT',
        'ARX/USDT',
        'DASH/USDT',
        'WAXP/USDT',
        'AI/USDT',
        'ONE/USDT',
        'ENJ/USDT',
        'BTT/USDT',
        'BCH/USDT',
        'PDEX/USDT',
        'NWC/USDT',
        'EWT/USDT',
        'FIL/USDT',
        'IOST/USDT',
        'XMR/USDT',
        'ZIL/USDT',
        'SXP/USDT',
        'CHZ/USDT',
        'MANA/USDT',
        'SUSHI/USDT',
        'GALAX/USDT',
        'UNI/USDT',
        'NEO/USDT',
        'CQT/USDT'
    ],
    days=365,
    starting_balance=100,
    stake_amount='unlimited',
    max_open_trades=3,
    interval='5m',
    inf_interval='30m 1h',
    tag='20210820-20210915'
)

In [ ]:
# QuickTools.download_data(us_conf, '30m', timerange='20200801-', pairs=['BTC/USD'], verbose=True)
QuickTools.download_data(kucoin, '30m', timerange='20200801-', pairs=['BTC/USDT'], verbose=True)

In [5]:
# print(b_commands[0].command_string)
b_commands = backtest.create_commands(
    b_params,
    verbose=False,
)
b_runner = backtest.BacktestRunner(b_commands[0])
b_runner.execute()
if b_runner.error:
    print(b_runner.output)
else:
    b_runner.save()
    get_backtest_repo().df().head(1)

''

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
0,NASOSv4,09/15/21 16:09:24,,binanceus,3,unlimited,100,4.08,17.68,0.28,-14.75,"-5,303.01",-1.63,-10.25,27.51,6,26,20210820-


In [ ]:
print(b_runner.output)

In [ ]:
b_commands = backtest.create_commands(
    b_params,
    verbose=False,
)
bm_runner = backtest.BacktestMultiRunner(b_commands)
bm_runner.execute()
if any(bm_runner.errors):
    print(bm_runner.errors)

bm_runner.save()
get_backtest_repo().df().head(len(bm_runner.reports))

In [ ]:
print(bm_runner.runners[0].output)

In [ ]:
get_backtest_repo()[0].trades_to_csv()

# Reports

## Hyperopt Reports

In [ ]:
get_hyperopt_repo().df()

## Backtest Reports

In [10]:
get_backtest_repo().delete(0)

In [ ]:
get_backtest_repo().df().sort_values('roiloss').head(10)

In [ ]:
(get_backtest_repo()
 .filter_by_strategy('NotAnotherSMAOffsetStrategyHOv3', 'NotAnotherSMAOffsetStrategyHOv3Mod', 'NotAnotherSMAOffsetStrategyModHO')
.df()
 .query('days>364')
 .sort_values(by=['strategy', 'score'], ascending=False))


In [ ]:
get_backtest_repo().filter_by_id('4db2236-0861-4d7b-b720-7ecd04999290')

In [ ]:
# import rapidjson

# rapidjson.loads(
#     ParameterTools.get_path_of_params(
#         "ca1f7c9b-e3d6-483b-b912-00688410f0ed"
#     ).read_text()
# )
get_hyperopt_repo().get_by_param_id('b40294bf-de3b-4f7d-a6f7-a412709e549c').parameters()

# Remote

In [ ]:
bot1 = remote.RemoteBot(1, "vps")
bot4 = remote.RemoteBot(4, "vps")
bot2 = remote.RemoteBot(2, "pi")
bot3 = remote.RemoteBot(3, "pi")
bot5 = remote.RemoteBot(5, "pi")

In [ ]:
bot4.tools.send_file(4, str(priv_us), "user_data/priv_us.json")

In [ ]:
c = remote.RemoteBotInfo(1).config
c.pairslist

In [ ]:
bot4.update_config(dict(max_open_trades=3))
bot4.restart()

In [ ]:
bot = bot5
bot.set_strategy("NotAnotherSMAOffsetStrategyHOv3Mod", id="b40294bf-de3b-4f7d-a6f7-a412709e549c")
# bot.set_run_mode("live")
bot.env
bot.restart()

In [ ]:
bot4.get_trades().to_csv('exports/bot4_trades.csv', index=False)

In [ ]:
bot2.get_trades().profit_abs.sum()

In [ ]:
bot3.env

In [ ]:
# bot3.restart()
# bot5.restart()
bot2.env

# Misc

In [ ]:
!freqtrade hyperopt-show

In [ ]:
!freqtrade hyperopt --help

In [ ]:
!freqtrade list-strategies

In [ ]:
!echo "hi"

# Logs

In [ ]:
!tail --help